In [ ]:
#%%
"""Import dependant modules"""

import datetime
import datetime as dt

from datetime import date, datetime, timedelta
from json import loads
import pandas as pd
import matplotlib
import matplotlib.cm as cm
import matplotlib.dates as mdate
import matplotlib.patches as mfatches
import matplotlib.pyplot as pt
import matplotlib.pyplot as plt
import matplotlib.pyplot as plf
import matplotlib.ticker as mticker
import numpy
import numpy as np
import plotly.io as plt_io

import seaborn as sns


"""# Breadth"""
# Turn this into a Class that I can import into the main program!
# Import `os`
today = date.today()
# print("Contact info: 403-463-8072")
# print("Email: joe@quantinvests.com")
try:
    import datetime
except:
    pass

date_types = (
    pd.Timestamp,
    pd.DatetimeIndex,
    pd.Period,
    pd.PeriodIndex,
    dt.datetime,
    dt.time
)

one_day = datetime.timedelta(days=1)
yesterday = today - one_day
tomorrow = today + one_day
tomorrow = today + one_day
start = datetime.datetime.now() - datetime.timedelta(days=720)
start_long = datetime.datetime.now() - datetime.timedelta(days=3500)
start_short = datetime.datetime.now() - datetime.timedelta(days=320)
start_min = datetime.datetime.now() - datetime.timedelta(days=59)
end = tomorrow

sns.set_style('whitegrid')

pd.core.common.is_list_like = pd.api.types.is_list_like

# this helps us get the theme settings
# Necessary imports:
import matplotlib

# this is for simple plotting with plotly express
style = matplotlib.style.use('dark_background')
plt.style.use('dark_background')
# create our custom_dark theme from the plotly_dark template
plt_io.templates["custom_dark"] = plt_io.templates["plotly_dark"]

# set the paper_bgcolor and the plot_bgcolor to a new color
plt_io.templates["custom_dark"]['layout']['paper_bgcolor'] = '#30404D'
plt_io.templates["custom_dark"]['layout']['plot_bgcolor'] = '#30404D'

# you may also want to change gridline colors if you are modifying background
plt_io.templates['custom_dark']['layout']['yaxis']['gridcolor'] = '#4f687d'
plt_io.templates['custom_dark']['layout']['xaxis']['gridcolor'] = '#4f687d'

In [ ]:
from google.colab import files
uploaded = files.upload()

import io
df2 = pd.read_csv(io.BytesIO(uploaded['OSC.csv']))
# Dataset is now stored in a Pandas Dataframe

# Breadth Download data from Mcclenan breadth data

Link: https://www.mcoscillator.com/market_breadth_data/

In [ ]:

"""Import Breadth Data"""
# import py
Breadth = pd.read_csv("OSC.csv",
                      index_col=0, parse_dates=True)

Breadth.dropna(inplace = True)
print(Breadth.tail(5))

Breadth['Close'] = Breadth.DJIA
Breadth["20d"] = np.round(Breadth["Close"].rolling(window=20, center=False).mean(), 2)
Breadth["50d"] = np.round(Breadth["Close"].rolling(window=50, center=False).mean(), 2)
Breadth["200d"] = np.round(Breadth["Close"].rolling(
    window=200, center=False).mean(), 2)

"""Above Displays a line chart of price and 3 SMAs"""
Breadth['Sell_Sig'] = ((Breadth['A-D'] < 148) &
                       (Breadth['A-D'].shift(1) > 150))
# Create a column in the DataFrame showing "TRUE" if sell exit signal is given and "FALSE" otherwise.
# A sell exit signal is given when the %K line crosses back up through the %D line
Breadth['Sell_Exit'] = ((Breadth['A-D'] > -150) &
                        (Breadth['A-D'].shift(1) < -150))

Breadth['Buy_Sig'] = ((Breadth['A-D'] > -150) &
                      (Breadth['A-D'].shift(1) < -150))
# Create a column in the DataFrame showing "TRUE" if sell exit signal is given and "FALSE" otherwise.
# A sell exit signal is given when the %K line crosses back up through the %D line
Breadth['Buy_Exit'] = (Breadth['A-D'] < 0)

"""Plot SMAs"""
Breadth[['Close', '20d', '50d', '200d']].plot(figsize =(15, 6))
fig, ax = plt.subplots(figsize=(15, 6))
a = Breadth.loc[Breadth['Sell_Sig'] == True, ['DJIA', 'A-D']]
b = Breadth.loc[Breadth['Buy_Sig'] == True, ['DJIA', 'A-D']]  # anomaly
Breadth['Short'] = np.nan
# Breadth = Breadth.fillna('0.0')
Breadth.loc[Breadth['Sell_Sig'], 'Short'] = -1.0
Breadth.loc[Breadth['Sell_Exit'], 'Short'] = 0.0
# Set initial MP on day 1 to flat
# Breadth['Short'][0] = 0.0
# Forward fill the MP column to represent the holding of MPs through time
Breadth['Short'] = Breadth['Short'].fillna(method='pad')
# Create a column in the DataFrame showing "TRUE" if buy entry signal is given and "FALSE" otherwise.
# A buy is initiated when the %K line crosses up through the %D line and the value of the oscillator is below 20
# create a placeholder column to polulate with long MPs (1 for long and 0 for flat) using boolean values created above
Breadth['Long'] = np.nan
Breadth.loc[Breadth['Buy_Exit'], 'Long'] = 0.0

# Set initial MP on day 1 to flat
# Breadth['Long'][0] = 0.0
# Forward fill the MP column to represent the holding of MPs through time
Breadth.loc[Breadth['Buy_Sig'], 'Long'] = 1.0
Breadth['Long'] = Breadth['Long'].fillna(method='pad')
# Add Long and Short MPs together to get final strategy MP (1 for long, -1 for short and 0 for flat)

Breadth['IM_POS'] = Breadth['Long'] + Breadth['Short']

ax = plt.subplot(212)
ax.plot(Breadth['A-D'], color='blue')
ax.scatter(a.index, a['A-D'], color='r')
ax.scatter(b.index, b['A-D'], color='g')
plt.axhline(y=150, color='w', linestyle='-')
plt.axhline(y=-150, color='w', linestyle='-')
ax1 = plt.subplot(211)
ax1.plot(Breadth['DJIA'], color='blue')
ax1.scatter(a.index, a['DJIA'], color='r')
ax1.scatter(b.index, b['DJIA'], color='g')
plt.show();

# ax = Breadth[['A-D']].plot(
#     figsize=(14, 4), marker='s', linestyle='-', style=['g--', 'r--'], lw=0.8)

Breadth['Daily_rtrns'] = (
    Breadth['DJIA'] - Breadth['DJIA'].shift(1)) / Breadth['DJIA'].shift(1) * 100

Breadth['Sys_rtrns'] = Breadth['IM_POS'] * Breadth['Daily_rtrns']

Breadth['Sys_cusum'] = Breadth['Sys_rtrns'].cumsum()
Breadth['SPY_cumsum'] = Breadth['Daily_rtrns'].cumsum()

Sharpe = (Breadth['Sys_cusum']-Breadth['SPY_cumsum']) / \
          np.nanstd(Breadth['SPY_cumsum'])

fig = plt.subplots(figsize=(15, 4))
plt.plot(Breadth['Sys_cusum'], color='g',
         label='IMO Breadth advance vs decline trade signals')
plt.plot(Breadth['SPY_cumsum'], color='r', label='Market Returns')
# plt.figtext(0.5,0.9, s='Sharpe ratio: %.2f'%Sharpe)
plt.legend(loc='best')
plt.show()
#%%
print(f"Sharpe ratio: {Sharpe[-1]}")

print("Number of Trades:")
print(18* "*")
print(Breadth[['IM_POS']].value_counts())
# df.to_sql('Breadth_{}'.format(str(today)), engine)
#%%